In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
# %pip install -U trl 
%pip install git+https://github.com/huggingface/trl.git
%pip install datasets==2.16.0

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, wandb, platform, warnings
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login
#Use a sharded model to fine-tune in the free version of Google Colab.
# base_model = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1" #bn22/Mistral-7B-Instruct-v0.1-sharded
base_model = "alexsherstinsky/Mistral-7B-v0.1-sharded"
dataset_name, new_model = "/kaggle/input/damir-gai", "bias_mistral_7b"

2024-04-29 15:01:13.345425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 15:01:13.345548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 15:01:13.474668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Preprocessing of the train data

In [3]:
import pandas as pd
# Create a list to store formatted entries
formatted_entries = []
df = pd.read_json('/kaggle/input/damir-gai/dl_project_train.json', encoding = 'utf-8')
# Iterate over each entry in the DataFrame
for index, row in df.iterrows():
    large_label = row['large_label']
    
    # Iterate over each article in the 'articles' field
    for article in row['articles']:
        # Extract title and content
        title = article['title']
        content = article['content']
#         if i == 0:
#             print(content)
#         i +=1
        words = content.split()
        if len(words) > 256:
            content = ' '.join(words[:256]) + '...'
        
        # Format the entry
        formatted_entry = f"Define main factuality and bias properties of the article. [INST] '{title}' \n '{content}' [/INST] {large_label}"
        formatted_entries.append(formatted_entry)

data_dict = {"text": formatted_entries}

# Create a Dataset object
dataset = Dataset.from_dict(data_dict)

# Print the dataset
print(dataset['text'][0])


Define main factuality and bias properties of the article. [INST] 'An Independent Voter Guide to The Impeachment Trial' 
 'The impeachment trial ofÂ President Donald J. Trump is underway! And it is... intensely predictable. This is why more and more voters are becoming independent. Democrats Want TrumpÂ RemovedÂ *GASP* And Republicans (You Guessed It)Â Want HimÂ Acquitted CNN's public polling finds voters are evenly divided between wanting the Senate to remove Trump from office, and wanting the Senate to acquit him.Â How curiously suspicious that Democrats overwhelmingly support removal and Republicans overwhelmingly support acquittal! What are the odds that most Americans' opinions about the certain, specific, localized actions of a few individuals, and how to legally interpret them, just happen to line up perfectly with their broad ideological and cultural commitments to an entire political party or the other? One can imagine a parallel universe in which the president on trial were a

# Config for QLoRA technique

In [4]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

(True, True)

# Login to weight and bias site where we stored all metrics and graphics

In [5]:
# !wandb login --relogin
wandb.login(key = "1c104e13860a873640d0053b9a5841f8ef83117a")
run = wandb.init(project='Fine tuning pol bias mistral', job_type="training", anonymous="allow")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: damiron4. Use `wandb login --relogin` to force relogin


# QLoRA

In [6]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

# Training Arguments

In [7]:

#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)


# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)


/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6994 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


# Train process

In [ ]:
trainer.train("/kaggle/input/gai_proj/pytorch/checkpoint-300/1/gai_new_checkpoint-300")
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
310,1.672800
320,1.753500
330,1.835800
340,1.827500
350,1.402400
360,1.679200
370,1.712500


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain curr

# TESTING (inference)

Here we load the original model

In [3]:
adapters_name = "/kaggle/input/gai_proj/pytorch/checkpoint-370/1/gai_new_checkpoint-370"
model_name = "alexsherstinsky/Mistral-7B-v0.1-sharded"
device = "cuda" # the device to load the model onto

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
#     model_name,
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token




config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

(True, True)

# Here we insert the learned adapter

In [8]:
model = PeftModel.from_pretrained(model, adapters_name)
model.eval()
print(f"Successfully loaded the model {model_name} into memory")

Successfully loaded the model alexsherstinsky/Mistral-7B-v0.1-sharded into memory


# Function for the inference

In [9]:
def stream(user_prompt, model, tokenizer):
    runtimeFlag = "cuda:0"
    B_INST, E_INST = "[INST] ", " [/INST] "
    system_prompt = 'Define main factuality and bias properties of the article. '
    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate output without streaming
    output = model.generate(**inputs, max_new_tokens=128)

    # Convert the generated output to a string
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    # Return the generated output in string format
    return output_str
    


# testing

In [12]:
title = "Not This Little Black Duck \u2013 Quadrant Online"
words = "There is much to admire about Bruce Pascoe. He is a successful author and seems to have immense physical and intellectual energy for a man of more than 70 years. It seems he was a stalwart in fighting the 2019 bushfires near his bush home, and good on him. But his current status as Australia\u2019s \u201cmost influential Indigenous historian\u201d, according to The Australian \u2019s Richard Guilliatt , and recently appointed Enterprise Professor of Indigenous Agriculture at Melbourne University is built on a lie, a dangerous lie at that. With his falsehoods now being foisted upon our school children, I intend to continue calling out his fictions as often as I can. So please bear with me.\n\nSpeaking in his professorial role, Pascoe last week told the ABC\u2019s Q&A audience that if we were to adopt the sophisticated agricultural and political practices of traditional Aborigines, we could feed ourselves with no harm to the environment. Bruce practices what he preaches on his property at Gypsy Point, near Mallacoota, and has established Black Duck Foods, a registered charity, to give effect to his vision. Its website tells us that \u201cBlack Duck Foods enjoys \u2018 deductible gift status\u2019 (DGR) status, registered as a Public Benevolent Institution with the Australian Not For Profit and Charities Commission\u201d. And that \u201cBlack Duck Foods welcomes the support of community members and volunteers to assist with our business, both on and off the farm.\u201d\n\nIts ambition: \u201cBlack Duck Foods has a vision to redevelop traditional food growing and country management processes for the economic benefit of Indigenous people and country.\u201d Never one to let a good bandwagon be other than fully occupied, Pascoe has incorporated the concept of \u2018sovereignty\u2019 into food production. It seems that Black Duck plans to assist Indigenous people on a journey towards \u2018food sovereignty\u2019 . What that means, apparently, is this:\n\nCentral to First Nations sovereignty is food sovereignty \u2013 the right to define one\u2019s own food system. The right to produce our own cultural food.\n\nI\u2019m not sure where and when that right has ever been curtailed. Many remote community Aborigines have maintained their traditional hunter\/gatherer lifestyle, albeit supplemented by a Western diet. Still, \u2018food sovereignty\u2019 has nice ring to it \u2013 if you\u2019re into mindless slogans, that is.\n\nBut back to Q&A , where Bruce managed to get in his usual plugs about Aboriginal society inventing government 120,000 years ago, was harmonious, settled and sophisticated. Following a passionate plea by Stan Grant for social justice for Aboriginal people, Pascoe delivered this gem of a rant :\n\nAnd it\u2019s not just social justice. Have a look at the country. Have a look at the length of time people were able to work together here and have a prosperous lifestyle over a longer period of time than any other people on earth and only interrupted by the British. How did they do that? Isn\u2019t this a question the world is interested in? It\u2019s not just about Aboriginal people getting justice \u2013 we\u2019re not a charity.\n\nThis was a vibrant successful society which was sustainable. Isn\u2019t the world interested in this? Because that was government here. This was government and it was done over an enormous period of time. I think this is a social jewel in the world and needs to be analysed. We hold something here and it\u2019s in the art. The Burrup Peninsula is being mined at the moment. Deliberately being damaged by people who are trying to put Aboriginal people in their place, ignoring that incredible art and that incredible culture.\n\nLet me introduce a note of reality here. \u2018A prosperous lifestyle over a longer period of time than any other people on earth\u2019? Does he think the Western nations or China, for example, were plonked here on earth by alien invaders? What he means is \u2018a lifestyle unchanged over a longer period of time than any other people on earth\u2019. Is that especially valuable, or is it no more than an anthropological curiosity?\n\nAs to the Burrup Peninsula in the Pilbara, it is indeed home to what is regarded as the most extensive collection of rock art in the world. But it is rudimentary art. The only thing \u201cincredible\u201d about it is that there is so much of it and that it has survived so long. The same can be said about the culture. That is not to say we shouldn\u2019t make every effort to preserve as much of it as possible. It is true there is a clash between the demands of mining and the preservation of Aboriginal art in the Pilbara, however the Murujuga National Park website tells us that:\n\nAs a result of the Burrup and Maitland Industrial Estates Agreement (BMIEA), Murujuga was declared a National Park on the 17th of January 2013, becoming the 100th National Park in Australia. The National Park consists of freehold land title in excess of 5,134ha, which is vested in MAC. This land title includes a 221ha block, known as \u2018Site L\u2019, which was handed back to MAC in July 2019 and added to the Murujuga National Park. The area of Murujuga National Park sits within the broader National Heritage Listed area which comprises the surrounding islands of the Dampier Archipelago \u2026 Access to some rock engraving sites within the park is restricted under Aboriginal lore and custom. The rock arts sites are also protected under the WA Aboriginal Heritage Act 1972, which sets penalties for disturbing or interfering with the sites.\n\nSo, Pascoe might argue that mining should defer to Aboriginal rock art and that everything must be preserved \u2013 a la the fake controversy over Jukaan Gorge \u2014 but he cannot say that miners are deliberately damaging rock art \u2018to put Aboriginal people in their place\u2019 . This kind of inflammatory rhetoric pervades Dark Emu by the way.\n\nWhen fellow guest, The Australian \u2018s Adam Creighton observed that colonisation was inevitable but also brought progress with it, Pascoe wasn\u2019t having any of it. The response of the man who once said \u201cI\u2019m not thoroughly convinced by capitalism. But while it\u2019s here and enslaving our people, we have to participate\u201d was more of the same: \u201cAdvancements are not monetary \u2014 advancements are moral\u201d. That went over like Pavlov\u2019s bell with the standard-issue ABC crowd of knee-jerk applauders, whose approving din almost drowned out Creighton\u2019s response: \u201cWhat about electricity?\u201d Had Creighton been given half a chance he might have expanded on the host of other benefits, starting with an abhorrence of infanticide, cannibalism and wife beating, notwithstanding that examples of the latter still abound.\n\nStan Grant, to his credit, observed that he values both cultures. He loves philosophy, he loves history, he loves economics \u2013 all of which would have cut no ice with Pascoe, who maintains that Aboriginal culture embodied all three. Stan also loves to read and he reveres Shakespeare. Give him time and Pascoe may well claim to have found in some explorer\u2019s journal an account of attending a first-night gala in the theatre of one of those permanent Aboriginal \u2018towns\u2019 he insists have been erased from the record by racist white revisionists.\n\nBut getting back to Bruce, he might be jumping the gun in claiming our current agricultural practices are destroying the environment, and that a charitable enterprise which relies on volunteers, donations and grants, is the answer.\n\nWhatever you might think about the depredations of current agricultural practices, which feed almost 8 billion people and continue to produce produce more food year-on-year, it\u2019s not all bad news. Let me introduce to you Green Camel , an organic farming operation on less than half a hectares just west of Sydney. You won\u2019t find it listed in any Charities Commission filings . What you will find is an operation that uses closed-system glasshouses to produce as much as 160,000kg of leafy greens, 16,000kg of tomatoes and 12,000kg of barramundi per year. It, too, started with seed funding from the government of NSW, but does not rely on volunteers or donations. As an example of modern Western technology and practice maximising output while minimising impact on the environment, it\u2019s hard to go past Green Camel.\n\nAt the glacial pace of traditional Aboriginal development, Black Duck Foods, which as far as I can tell has so far baked a few loaves of native-seed bread, might achieve this level of output in, say, the year 2200. What won\u2019t be lacking in the meantime is loads of the metaphoric fertiliser that Pascoe is so adept at spreading.\n\nYou can order the new edition of Peter O\u2019Brien\u2019s Bitter Harvest by clicking her e"


prompt = f"'{title}'\n'{words}'"



ans = stream(prompt, model, tokenizer)
print(ans)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Define main factuality and bias properties of the article. [INST] 'Not This Little Black Duck – Quadrant Online'
'There is much to admire about Bruce Pascoe. He is a successful author and seems to have immense physical and intellectual energy for a man of more than 70 years. It seems he was a stalwart in fighting the 2019 bushfires near his bush home, and good on him. But his current status as Australia’s “most influential Indigenous historian”, according to The Australian ’s Richard Guilliatt , and recently appointed Enterprise Professor of Indigenous Agriculture at Melbourne University is built on a lie, a dangerous lie at that. With his falsehoods now being foisted upon our school children, I intend to continue calling out his fictions as often as I can. So please bear with me.

Speaking in his professorial role, Pascoe last week told the ABC’s Q&A audience that if we were to adopt the sophisticated agricultural and political practices of traditional Aborigines, we could feed oursel

# For untrained model (how we generated data for evaluating the base model)
Ex 1

In [14]:
import pandas as pd
# Create a list to store formatted entries
# formatted_entries = []
# labels = []
# a = 0
df = pd.read_json('/kaggle/input/damir-gai/dl_project_test.json', encoding = 'utf-8')
# Iterate over each entry in the DataFrame
count = 0
with open("gai_test_f1_res_untrained_exp2.txt", 'a') as f:
    for index, row in df.iterrows():
        large_label = row['large_label']
#         large_label = large_label if large_label is not None else ''
        large_label = large_label or ''

        count_articles = 0
        print("New website")
        # Iterate over each article in the 'articles' field
        for article in row['articles']:
            
            if count_articles > 2:
                break
            count += 1
            
            my_set = set()
#             if a == 5:
#                 break
            # Extract title and content
#             labels.append(large_label)
            title = article['title']
            content = article['content']
            words = content.split()
            words_256 = content.split()
            if len(words) > 256:
                words_256 = ' '.join(words[:256]) + '...'
            else:
                words_256 = ' '.join(words) + '...' 
#             words = ' '.join(words) + '...'
#             print(words)
            prompt = f"'{title}' \n '{words_256}'"
            ans = stream(prompt, model, tokenizer)
#             print(ans)
            predict = ''
            for t, i in enumerate(ans.split('[/INST]')):
                if t == 0:
                    continue
                predict += i.strip() + ' '
            split_pred = predict.split('\n')
            clean_text = " ".join(split_pred)

            # Convert set_to_str to an empty string if it is None

            each_row = large_label + '\t' + clean_text.strip() + '\n'
            print(each_row)
            f.write(each_row)
            count_articles += 1
            print(count)
#             a += 1
#         if i == 0:
#             print(content)
#         i +=1
#         words = content.split()
#         if len(words) > 256:
#             content = ' '.join(words[:256]) + '...'
        
        # Format the entry
#         formatted_entry = f"Define main factuality and bias properties of the article. [INST] '{title}' \n '{content}' [/INST] {large_label}"
#         formatted_entries.append(formatted_entry)

# data_dict = {"text": formatted_entries}

# Create a Dataset object
# dataset = Dataset.from_dict(data_dict)

# Print the dataset
# print(dataset['text'][0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


New website


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Journal Gazette - Least Biased - Credible - Reliable	ю 1. 2. 3. 4. 5. 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34.

1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

C-Span - Least Biased - Not Right Bias - Not Left Bias - Credible	ю  ### "Get 15% discount on your first 3 orders with us" Use the following coupon FIRST15  Order Now

294
New website


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Free Republic - Right Bias - Conservative - Republican - Credible	ю 2020-12-15 15:00:00.0  ### Related News  - #### McConnell congratulates Biden “The Electoral College has spoken..”  WNYW News New York, 15.12.2020, 15:00 - #### McConnell congratulates Biden “The Electoral College has spoken..”  Fox 5 New York, 15.12.2020, 15:00 - #### McConn

295


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Free Republic - Right Bias - Conservative - Republican - Credible	ю 2021-04-11 15:00:00.0  ### 1 Answer  - 1. The article is written in a neutral tone. 2. The article is factual. 3. The article is biased. 4. The article is biased. 5. The article is factual. 6. The article is biased. 7. The article is factual. 8. The article is biased. 9. The article is factual. 10. The article is biased. 1

296
Free Republic - Right Bias - Conservative - Republican - Credible	ю 1. 2. 3. 4. 5. 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34.

297


# For fine tuned model (how we generated data for evaluating the fine-tuned model)

In [16]:
import pandas as pd
# Create a list to store formatted entries
# formatted_entries = []
# labels = []
# a = 0
df = pd.read_json('/kaggle/input/damir-gai/dl_project_test.json', encoding = 'utf-8')
# Iterate over each entry in the DataFrame
count = 0
with open("gai_test_f1_res_untrained_exp1.txt", 'a') as f:
    for index, row in df.iterrows():
        large_label = row['large_label']
#         large_label = large_label if large_label is not None else ''
        large_label = large_label or ''

        count_articles = 0
        print("New website")
        # Iterate over each article in the 'articles' field
        for article in row['articles']:
            
            if count_articles > 2:
                break
            count += 1
            
            my_set = set()
#             if a == 5:
#                 break
            # Extract title and content
#             labels.append(large_label)
            title = article['title']
            content = article['content']
            words = content.split()
            words = ' '.join(words) + '...'
#             print(words)
            prompt = f"'{title}' \n '{words}'"
            ans = stream(prompt, model, tokenizer)
#             print(ans)
            predict = ''
            for t, i in enumerate(ans.split('[/INST]')):
                if t == 0:
                    continue
                if '-' in i:
                    predict += i + ' '
            for i in predict.split('-'):
                my_set.add(i)
            set_to_str = ('- '.join(my_set)).strip()
            
            # Convert set_to_str to an empty string if it is None
            set_to_str = set_to_str or ''
            each_row = large_label + '\t' + set_to_str + '\n'
            f.write(each_row)
            count_articles += 1
            print(count)
#             a += 1
#         if i == 0:
#             print(content)
#         i +=1
#         words = content.split()
#         if len(words) > 256:
#             content = ' '.join(words[:256]) + '...'
        
        # Format the entry
#         formatted_entry = f"Define main factuality and bias properties of the article. [INST] '{title}' \n '{content}' [/INST] {large_label}"
#         formatted_entries.append(formatted_entry)

# data_dict = {"text": formatted_entries}

# Create a Dataset object
# dataset = Dataset.from_dict(data_dict)

# Print the dataset
# print(dataset['text'][0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


New website


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

294
New website


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


295


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


296
297


# Clear memory

In [ ]:
# Clear the memory footprint
del model, trainer
torch.cuda.empty_cache()

# Reload the base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model_reload, new_model)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"